# Build a simple LLM application with chat models and prompt templates
This application will translate text from English into another language.
## Content
1. Setup
2. LangSmith
3. Using Language Models
4. Streaming
5. Prompt Templates

# 1. Setup

In [ ]:
%pip install langchain

# 2. LangSmith

In [1]:
# For Tracing (LangSmith)
# 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

In [2]:
import os
import getpass

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_API_KEY'] = getpass.getpass()

# 3. Using Language Models

In [2]:
%pip install -qU "langchain[groq]"

Note: you may need to restart the kernel to use updated packages.


In [3]:
# if not os.environ.get('GROQ_API_KEY'):
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model("llama3-8b-8192", model_provider='groq')

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("Translate the following from English to French"),
    HumanMessage("Hi! Do you speak English?")
]

model.invoke(messages)

AIMessage(content='Bonjour ! Parlez-vous anglais ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 29, 'total_tokens': 37, 'completion_time': 0.006666667, 'prompt_time': 0.004062394, 'queue_time': 0.018330713999999998, 'total_time': 0.010729061}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e93dfc0-b7eb-4efa-b1a6-006039fa5524-0', usage_metadata={'input_tokens': 29, 'output_tokens': 8, 'total_tokens': 37})

TIPS:

Nếu chúng ta đã bật LangSmith, chúng ta có thể thấy rằng lần chạy này được ghi vào LangSmith và có thể thấy dấu vết LangSmith. Dấu vết LangSmith báo cáo thông tin sử dụng mã thông báo, độ trễ, các tham số mô hình chuẩn (như nhiệt độ) và các thông tin khác.

In [14]:
# LangChain also supports chat model inputs via strings or OpenAI format. The following are equivalent:
print(f'Response 1: {model.invoke("Hello!")}')
print(f'Response 2: {model.invoke([{'role': 'user', 'content': 'Hello!'}])}')
print(f'Response 3: {model.invoke([HumanMessage('Hello!')])}')

Response 1: content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38, 'completion_time': 0.021666667, 'prompt_time': 0.001823676, 'queue_time': 0.020508483, 'total_time': 0.023490343}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None} id='run-1560e71f-3da3-41be-889e-8634026f97f2-0' usage_metadata={'input_tokens': 12, 'output_tokens': 26, 'total_tokens': 38}
Response 2: content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38, 'completion_time': 0.021666667, 'prompt_time': 0.001836466, 'queue_time': 0.017368733, 'total_time': 0.023503133}, 'model_name': 'llama3-8b-8192', 'system_fingerpri

# 4. Streaming

Vì các mô hình trò chuyện là Runnables, chúng hiển thị một giao diện chuẩn bao gồm các chế độ gọi async và streaming. Điều này cho phép chúng ta stream các token riêng lẻ từ một mô hình trò chuyện:

In [15]:
for token in model.stream(messages):
    print(token.content, end='|')

|Bonjour| !| Par|lez|-vous| anglais| ?||

# 5. Prompt Templates

- Prompt (lời nhắc/thông điệp)
- Prompt là những thông điệp mà chúng ta đưa vào mô hình
- Prompt được xây dựng bằng cách kết hợp đầu vào của người dùng và logic ứng dụng
- Logic ứng dụng này thường lấy đầu vào thô của người dùng và chuyển đổi nó thành một danh sách các thông điệp sẵn sàng để truyền đến mô hình
- Cách chuyển đổi phổ biến: thêm một thông điệp hệ thống hoặc định dạng một mẫu với đầu vào của người dùng

Mẫu nhắc nhở (prompt templates) là một khái niệm trong LangChain được thiết kế để hỗ trợ quá trình chuyển đổi này. Chúng tiếp nhận dữ liệu thô từ người dùng và trả về dữ liệu (lời nhắc) đã sẵn sàng để chuyển vào mô hình ngôn ngữ.

Let's create a prompt template here. It will take in two user variables:

    - language: The language to translate text into
    - text: The text to translate

In [29]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

- ChatPromptTemplate có thể hỗ trợ nhiều vai trò trong một template
- truyền vào một tham số language thành tin nhắn hệ thống (SystemMessage) và văn bản người dùng thành tin nhắn người dùng (text). Chúng ta có thể thử nghiệm các mẫu khác nhau để xem nó có thể tự làm gì!

In [37]:
prompt = prompt_template.invoke({"language": "French", "text": "Hi!"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Xin chào!', additional_kwargs={}, response_metadata={})])

Chúng ta có thể thấy rằng nó trả về một ChatPromptValue bao gồm hai tin nhắn. Nếu chúng ta muốn truy cập trực tiếp vào các tin nhắn, chúng ta thực hiện:

In [38]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Xin chào!', additional_kwargs={}, response_metadata={})]

In [40]:
response = model.invoke(prompt)
print(f'Response: {response}')
print(f'Content: {response.content}')

Response: content='Bonjour!' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 27, 'total_tokens': 30, 'completion_time': 0.0025, 'prompt_time': 0.003916825, 'queue_time': 0.019110424, 'total_time': 0.006416825}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None} id='run-21406647-18b6-4ecd-bf96-9376447b1343-0' usage_metadata={'input_tokens': 27, 'output_tokens': 3, 'total_tokens': 30}
Content: Bonjour!
